In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

data = pd.read_csv('../input/happy-whale-and-dolphin/train.csv')

classes = data['individual_id'].unique()

print(len(data['individual_id'].unique()))
#print(data.shape)

In [ ]:
#making folders for each label

os.mkdir('/kaggle/data')
os.mkdir('/kaggle/data/train_data')
for i in classes:
    s = '/kaggle/data/train_data/'+str(i)
    os.mkdir(s)
        

In [ ]:
import shutil

for i in range(len(data)):
    #print(data.iloc[i])
    s1 = '../input/whale2-cropped-dataset/cropped_train_images/cropped_train_images'+'/'+data.iloc[i]['image']
    s2 = '/kaggle/data/train_data/'+data.iloc[i]['individual_id']+'/'+data.iloc[i]['image']
    shutil.copy(s1,s2)
    

In [ ]:
import pandas as pd
import numpy as np
import shutil
import tensorflow as tf
from tensorflow import keras
from keras import Sequential, Model
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Conv2D, Flatten, BatchNormalization, Activation,MaxPooling2D
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [ ]:
base_model = tf.keras.applications.EfficientNetB6(weights='imagenet')

x = base_model.layers[-2].output
#pred = Dense(units=4096, activation='relu')(x)
pred1 = Dense(units=15587, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=pred1)

for layer in model.layers[:-2]:
    layer.trainable = False
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

ReduceLROnPlateau = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor= 0.1, min_lr=0.00001)



In [ ]:
gen = ImageDataGenerator( preprocessing_function=keras.applications.efficientnet.preprocess_input,
                          horizontal_flip = True, 
                          rotation_range=10, 
                          width_shift_range=0.2,
                          height_shift_range=0.2,
                          shear_range=0.2,
                          brightness_range=(0.7,1.3))

train_batches = gen.flow_from_directory(directory='/kaggle/data/train_data',target_size=(528,528) , batch_size=100, interpolation='bicubic')
valid_batches = ImageDataGenerator(preprocessing_function=keras.applications.efficientnet.preprocess_input).flow_from_directory(directory='/kaggle/data/train_data',target_size=(528,528) , batch_size=100, interpolation='bicubic')

In [ ]:
model.fit_generator(train_batches, epochs=5, callbacks=[ReduceLROnPlateau], steps_per_epoch=510, validation_data = valid_batches, validation_steps=50)

In [ ]:
model.save('effb6_phalanx_dataset.h5')